<a href="https://colab.research.google.com/github/saharkoohfar/EV-charging-load-forecasting/blob/main/Hybrid-BilLSTM-Embedding-Denoising-Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import train_test_split
import time
import numpy as np
import pandas as pd
import os 
from pandas import concat
import time
from math import sqrt
from numpy import concatenate
import matplotlib.pyplot as plt
from pandas import read_csv
from pandas import DataFrame
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

import IPython
import IPython.display
import matplotlib as mpl
import math

import argparse
import keras.callbacks as kc
from keras.layers import LSTM, GRU, Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Flatten
from keras.layers import Bidirectional
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dense, SimpleRNN
import tensorflow as tf
import itertools

import torch
from sklearn.model_selection import train_test_split
import time

from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

from itertools import product
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
 
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
mpl.rcParams['figure.figsize'] = (15, 8)
mpl.rcParams['axes.grid'] = False



In [ ]:
torch.manual_seed(5645)
import random
random.seed(5645)
np.random.seed(5645)

In [ ]:
data = pd.read_csv('HourlyData.csv')
data.set_index('date_from', inplace=True)
data.index = pd.to_datetime(data.index)
data

,SUM,daily_hour
date_from,,
2018-12-21 10:00:00,2.34,10
2018-12-21 11:00:00,4.86,11
2018-12-21 12:00:00,10.80,12
2018-12-21 13:00:00,10.80,13
2018-12-21 14:00:00,10.80,14
...,...,...
2020-01-31 18:00:00,27.27,18
2020-01-31 19:00:00,33.37,19
2020-01-31 20:00:00,9.94,20


Transformer

In [ ]:

def split_dataset_into_seq(dataset, start_index=0, end_index=None, history_size=240, step=1):
    '''split the dataset to have sequence of observations of length history size'''
    dataset = np.array(dataset)
    data = []
    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset)
    for i in range(start_index, end_index):
        indices = range(i - history_size, i, step)
        data.append(dataset[indices])
    return np.array(data)

def split_dataset(data, TRAIN_SPLIT=0.7, VAL_SPLIT=0.5, save_path=None):
    '''split the dataset into train, val and test splits'''
    # normalization

    data_mean = data.mean(axis=0)
    data_std = data.std(axis=0)
    data = (data - data_mean) / data_std
    stats = (data_mean, data_std)
   
    data_in_seq = split_dataset_into_seq(data, start_index=0, end_index=None, history_size=240, step=1)

    # split between validation dataset and test set:
    train_data, val_data = train_test_split(data_in_seq, train_size=TRAIN_SPLIT, shuffle=True, random_state=123)
    val_data, test_data = train_test_split(val_data, train_size=VAL_SPLIT, shuffle=True, random_state=123)
    return train_data, val_data, test_data, stats


def split_fn(chunk):
    """to split the dataset sequences into input and targets sequences"""

    chunk = chunk.astype(np.float)
    inputs = torch.from_numpy(chunk)
    targets = torch.from_numpy(chunk[:, :, 0]).unsqueeze(-1)

    return inputs, targets


def data_to_dataset(train_data, val_data, test_data, batch_size=64, target_features=list(range(2))):
    '''
    split each train split into inputs and targets
    convert each train split into a tf.dataset
    '''
    x_train, y_train = split_fn(train_data)
    x_val, y_val = split_fn(val_data)
    x_test, y_test = split_fn(test_data)
    train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
    val_dataset = torch.utils.data.TensorDataset(x_val, y_val)
    test_dataset = torch.utils.data.TensorDataset(x_test, y_test)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)
    return train_loader, val_loader, test_loader


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_data, val_data, test_data, stats = split_dataset(data)
train_dataset, val_dataset, test_dataset = data_to_dataset(train_data, val_data, test_data)

"""Implementation of the Transformer model"""

import torch.nn as nn
import math


class MultiHeadAttention(nn.Module):
    '''Multi-head self-attention module'''

    def __init__(self, D, H):
        super(MultiHeadAttention, self).__init__()
        self.H = H
        self.D = D

        self.wq = nn.Linear(D, D * H)
        self.wk = nn.Linear(D, D * H)
        self.wv = nn.Linear(D, D * H)

        self.dense = nn.Linear(D * H, D)

    def concat_heads(self, x):
        '''(B, H, S, D) => (B, S, D*H)'''
        B, H, S, D = x.shape
        x = x.permute((0, 2, 1, 3)).contiguous()  # (B, S, H, D)
        x = x.reshape((B, S, H * D))  # (B, S, D*H)
        return x

    def split_heads(self, x):
        '''(B, S, D*H) => (B, H, S, D)'''
        B, S, D_H = x.shape
        x = x.reshape(B, S, self.H, self.D)  # (B, S, H, D)
        x = x.permute((0, 2, 1, 3))  # (B, H, S, D)
        return x

    def forward(self, x, mask):
        q = self.wq(x)  # (B, S, D*H)
        k = self.wk(x)  # (B, S, D*H)
        v = self.wv(x)  # (B, S, D*H)

        q = self.split_heads(q)  # (B, H, S, D)
        k = self.split_heads(k)  # (B, H, S, D)
        v = self.split_heads(v)  # (B, H, S, D)

        attention_scores = torch.matmul(q, k.transpose(-1, -2))  # (B,H,S,S)
        attention_scores = attention_scores / math.sqrt(self.D)

        # add the mask to the scaled tensor.
        if mask is not None:
            attention_scores += (mask * -1e9)

        attention_weights = nn.Softmax(dim=-1)(attention_scores)
        scaled_attention = torch.matmul(attention_weights, v)  # (B, H, S, D)
        concat_attention = self.concat_heads(scaled_attention)  # (B, S, D*H)
        output = self.dense(concat_attention)  # (B, S, D)

        return output, attention_weights


# Positional encodings
def get_angles(pos, i, D):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(D))
    return pos * angle_rates


def positional_encoding(D, position=100, dim=3, device=device):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                            np.arange(D)[np.newaxis, :],
                            D)
    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    if dim == 3:
        pos_encoding = angle_rads[np.newaxis, ...]
    elif dim == 4:
        pos_encoding = angle_rads[np.newaxis, np.newaxis, ...]
    return torch.tensor(pos_encoding, device=device)


# function that implement the look_ahead mask for masking future time steps.
def create_look_ahead_mask(size, device=device):
    mask = torch.ones((size, size), device=device)
    mask = torch.triu(mask, diagonal=1)
    return mask  # (size, size)



class TransformerLayer(nn.Module):
    def __init__(self, D, H, hidden_mlp_dim, dropout_rate):
        super(TransformerLayer, self).__init__()
        self.dropout_rate = dropout_rate
        self.mlp_hidden = nn.Linear(D, hidden_mlp_dim)
        self.mlp_out = nn.Linear(hidden_mlp_dim, D)
        self.layernorm1 = nn.LayerNorm(D, eps=1e-9)
        self.layernorm2 = nn.LayerNorm(D, eps=1e-9)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.mha = MultiHeadAttention(D, H)

    def forward(self, x, look_ahead_mask):
        attn, attn_weights = self.mha(x, look_ahead_mask)  # (B, S, D)
        attn = self.dropout1(attn)  # (B,S,D)
        attn = self.layernorm1(attn + x)  # (B,S,D)

        mlp_act = torch.relu(self.mlp_hidden(attn))
        mlp_act = self.mlp_out(mlp_act)
        mlp_act = self.dropout2(mlp_act)

        output = self.layernorm2(mlp_act + attn)  # (B, S, D)

        return output, attn_weights



class Transformer(nn.Module):
    '''Transformer Decoder Implementating several Decoder Layers.
    '''

    def __init__(self, num_layers, D, H, hidden_mlp_dim, inp_features, out_features, dropout_rate, pred_len, embed_model ):
        super(Transformer, self).__init__()
        self.sqrt_D = torch.tensor(math.sqrt(D))
        self.num_layers = num_layers
        self.embed_model = embed_model
        if embed_model == 'LSTM':
          self.input_projection = nn.LSTM(inp_features, D)  # multivariate input
        else:
          self.input_projection = nn.Linear(inp_features, D)
        self.fitureSelection = nn.Linear(inp_features, inp_features) #feature selection
        self.output_projection = nn.Linear(D, out_features)  # multivariate output
        self.pos_encoding = positional_encoding(D)
        self.dec_layers = nn.ModuleList([TransformerLayer(D, H, hidden_mlp_dim,
                                                          dropout_rate=dropout_rate
                                                          ) for _ in range(num_layers)])
        self.dropout = nn.Dropout(dropout_rate)
        self.pred_len = pred_len
        self.D = D

    def forward(self, x, mask):
        B, S, D = x.shape
        attention_weights = {}
        x = nn.GELU()(self.fitureSelection(x))
        if self.embed_model == 'LSTM':
          hidden = torch.zeros((1, B, self.D), dtype=torch.double)
          x, _ = self.input_projection(x.permute(1, 0, 2), (hidden, hidden))
          x = x.permute(1, 0, 2)
        else:
          x= self.input_projection(x)

        x += self.pos_encoding[:, :S, :]
        x *= self.sqrt_D
        x = self.dropout(x)

        for i in range(self.num_layers):
            x, block = self.dec_layers[i](x=x,
                                          look_ahead_mask=mask)
            attention_weights['decoder_layer{}'.format(i + 1)] = block

        x = self.output_projection(x)
        x = x[:, -self.pred_len:, :]

        return x, attention_weights  # (B,S,S)


# Test Forward pass on the Transformer:
pred_len = 24

#creating list
pred_len = 24

#creating list
combinations = [[3,64,1,100],[1,64,1,10],[3,32,1,50],[1,64,1,50],[3,32,1,10]]

from tqdm import tqdm

conf_ls = {}

for i, config in enumerate(combinations):
    
    n_layers, d_model, head, epoch = config
    niter = len(train_dataset)
    losses, val_losses = [], []

    transformer = Transformer(num_layers=n_layers, D=d_model, H=head, hidden_mlp_dim=d_model, 
                              inp_features=2, out_features=1, dropout_rate=0.1, pred_len=pred_len, embed_model = 'Linear').double()
    transformer.to(device)

    optimizer = torch.optim.RMSprop(transformer.parameters(),
                                lr=0.00005)

    best_model = transformer

    val_loss = 1e5

    for e in tqdm(range(epoch)):
        start_time = time.time()
        # one epoch on train set
        transformer.train()
        sum_train_loss = 0.0
        for x, y in train_dataset:

            S = x.shape[1]
            y = y[:, 0:pred_len, :]
            mask = create_look_ahead_mask(S)
            out, _ = transformer(x, mask)
            loss = torch.nn.MSELoss()(out, y)
            sum_train_loss += loss.item()
            loss.backward()
            optimizer.step()
        losses.append(sum_train_loss / niter)

        # Evaluate on val set
        transformer.eval()
        sum_val_loss = 0.0
        for i, (x, y) in enumerate(val_dataset):
            S = x.shape[1]
            mask = create_look_ahead_mask(S)
            out, _ = transformer(x, mask)
            loss = torch.nn.MSELoss()(out, y)
            sum_val_loss += loss.item()
            if sum_val_loss < val_loss:
              val_loss = sum_val_loss
              best_model = transformer
            
        val_losses.append(sum_val_loss / (i + 1))

    test_losses, test_preds = [], []
    transformer.eval()
    for (x, y) in test_dataset:
        S = x.shape[-2]
        y_pred, _ = transformer(x,
                                mask=create_look_ahead_mask(S))
        loss_test = torch.nn.MSELoss()(y_pred, y)  # (B,S)
        test_losses.append(loss_test.item())
        test_preds.append(y_pred.detach().cpu().numpy())
    
    test_preds = np.vstack(test_preds)
    np.mean(test_losses)

    _, y_test = test_dataset.dataset.tensors

    y_test = y_test[1, :, 0].cpu().numpy()
    pred = test_preds[1, :, 0]

    MSE = np.square(np.subtract(y_test,pred)).mean() 
    RMSE = math.sqrt(MSE) #RMSE

    MAE = np.mean(np.abs(pred - y_test))    # MAE

    conf_ls["{},{},{},{}".format(n_layers, d_model, head, epoch)] = (RMSE, MAE)
    end_time = time.time()
    runtime = end_time - start_time
    print("Runtime: ", runtime, "seconds")

def RMSELoss(y_pred, y):
    return torch.sqrt(torch.mean((y_pred - y) ** 2))


criterion = RMSELoss
loss = criterion(y_pred, y)



def MAPELoss(y_pred, y):
    return torch.mean(torch.abs((y - y_pred) / y))


criterion = MAPELoss
loss = criterion(y_pred, y)


x = np.linspace(1, pred_len, pred_len)

data_mean, data_std = stats 
pred_T = test_preds*data_std[0]+ data_mean[0]
y = y_test*data_std[0] + data_mean[0]

print(conf_ls)
plt.plot(y, color='blue',marker='o', markerfacecolor='blue', markersize=5,)
plt.plot(pred_T[1], alpha=0.7, marker = '+',markersize=5, color='orange', linewidth = '5')



  0%|          | 0/100 [00:00<?, ?it/s]


RuntimeError: ignored

Denoising Autoencoder-Bilstm Transformer

In [ ]:

def split_dataset_into_seq(dataset, start_index=0, end_index=None, history_size=240, step=1):
    '''split the dataset to have sequence of observations of length history size'''
    dataset = np.array(dataset)
    data = []
    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset)
    for i in range(start_index, end_index):
        indices = range(i - history_size, i, step)
        data.append(dataset[indices])
    return np.array(data)

def split_dataset(data, TRAIN_SPLIT=0.7, VAL_SPLIT=0.5, save_path=None):
    '''split the dataset into train, val and test splits'''
    # normalization

    data_mean = data.mean(axis=0)
    data_std = data.std(axis=0)
    data = (data - data_mean) / data_std
    stats = (data_mean, data_std)
   
    data_in_seq = split_dataset_into_seq(data, start_index=0, end_index=None, history_size=240, step=1)

    # split between validation dataset and test set:
    train_data, val_data = train_test_split(data_in_seq, train_size=TRAIN_SPLIT, shuffle=True, random_state=123)
    val_data, test_data = train_test_split(val_data, train_size=VAL_SPLIT, shuffle=True, random_state=123)
    return train_data, val_data, test_data, stats


def split_fn(chunk):
    """to split the dataset sequences into input and targets sequences"""

    chunk = chunk.astype(np.float)
    inputs = torch.from_numpy(chunk)
    targets = torch.from_numpy(chunk[:, :, 0]).unsqueeze(-1)

    return inputs, targets


def data_to_dataset(train_data, val_data, test_data, batch_size=64, target_features=list(range(2))):
    '''
    split each train split into inputs and targets
    convert each train split into a tf.dataset
    '''
    x_train, y_train = split_fn(train_data)
    x_val, y_val = split_fn(val_data)
    x_test, y_test = split_fn(test_data)
    train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
    val_dataset = torch.utils.data.TensorDataset(x_val, y_val)
    test_dataset = torch.utils.data.TensorDataset(x_test, y_test)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)
    return train_loader, val_loader, test_loader


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_data, val_data, test_data, stats = split_dataset(data)
train_dataset, val_dataset, test_dataset = data_to_dataset(train_data, val_data, test_data)

"""Implementation of the Transformer model"""

import torch.nn as nn
import math

class DenoisingAutoencoder(nn.Module):
    def __init__(self, D):
        super(DenoisingAutoencoder, self).__init__()

        self.encoder = nn.Linear(D, 4*D)
        self.decoder = nn.Linear(4*D, D)
        self.relu= nn.ReLU()

    def forward(self, x):

        x_noisy = x + torch.randn_like(x) * 0.1  # add Gaussian noise
        x_encoded = self.relu(self.encoder(x_noisy))
        x_decoded = self.decoder(x_encoded)
        return x_decoded
        
class MultiHeadAttention(nn.Module):
    '''Multi-head self-attention module'''

    def __init__(self, D, H):
        super(MultiHeadAttention, self).__init__()
        self.H = H
        self.D = D

        self.wq = nn.Linear(D, D * H)
        self.wk = nn.Linear(D, D * H)
        self.wv = nn.Linear(D, D * H)

        self.dense = nn.Linear(D * H, D)
        self.dropout = nn.Dropout(0.1)
        
    def concat_heads(self, x):
        '''(B, H, S, D) => (B, S, D*H)'''
        B, H, S, D = x.shape
        x = x.permute((0, 2, 1, 3)).contiguous()  # (B, S, H, D)
        x = x.reshape((B, S, H * D))  # (B, S, D*H)
        return x

    def split_heads(self, x):
        '''(B, S, D*H) => (B, H, S, D)'''
        B, S, D_H = x.shape
        x = x.reshape(B, S, self.H, self.D)  # (B, S, H, D)
        x = x.permute((0, 2, 1, 3))  # (B, H, S, D)
        return x

    def forward(self, x, mask):
        q = self.wq(x)  # (B, S, D*H)
        k = self.wk(x)  # (B, S, D*H)
        v = self.wv(x)  # (B, S, D*H)

        q = self.split_heads(q)  # (B, H, S, D)
        k = self.split_heads(k)  # (B, H, S, D)
        v = self.split_heads(v)  # (B, H, S, D)

        attention_scores = torch.matmul(q, k.transpose(-1, -2))  # (B,H,S,S)
        attention_scores = attention_scores / math.sqrt(self.D)

        # add the mask to the scaled tensor.
        if mask is not None:
            attention_scores += (mask * -1e9)

        attention_weights = nn.Softmax(dim=-1)(attention_scores)
        scaled_attention = torch.matmul(attention_weights, v)  # (B, H, S, D)
        concat_attention = self.concat_heads(scaled_attention)  # (B, S, D*H)
        output = self.dense(concat_attention)  # (B, S, D)

        return output, attention_weights


# Positional encodings
def get_angles(pos, i, D):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(D))
    return pos * angle_rates


def positional_encoding(D, position=100, dim=3, device=device):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                            np.arange(D)[np.newaxis, :],
                            D)
    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    if dim == 3:
        pos_encoding = angle_rads[np.newaxis, ...]
    elif dim == 4:
        pos_encoding = angle_rads[np.newaxis, np.newaxis, ...]
    return torch.tensor(pos_encoding, device=device)


# function that implement the look_ahead mask for masking future time steps.
def create_look_ahead_mask(size, device=device):
    mask = torch.ones((size, size), device=device)
    mask = torch.triu(mask, diagonal=1)
    return mask  # (size, size)



class TransformerLayer(nn.Module):
    def __init__(self, D, H, hidden_mlp_dim, dropout_rate):
        super(TransformerLayer, self).__init__()
        self.dropout_rate = dropout_rate
        self.mlp_hidden = nn.Linear(D, hidden_mlp_dim)
        self.mlp_out = nn.Linear(hidden_mlp_dim, D)
        self.layernorm1 = nn.LayerNorm(D, eps=1e-9)
        self.layernorm2 = nn.LayerNorm(D, eps=1e-9)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.mha = MultiHeadAttention(D, H)

    def forward(self, x, look_ahead_mask):
        attn, attn_weights = self.mha(x, look_ahead_mask)  # (B, S, D)
        attn = self.dropout1(attn)  # (B,S,D)
        attn = self.layernorm1(attn + x)  # (B,S,D)

        mlp_act = torch.relu(self.mlp_hidden(attn))
        mlp_act = self.mlp_out(mlp_act)
        mlp_act = self.dropout2(mlp_act)

        output = self.layernorm2(mlp_act + attn)  # (B, S, D)

        return output, attn_weights



class Transformer(nn.Module):
    '''Transformer Decoder Implementating several Decoder Layers.
    '''

    def __init__(self, num_layers, D, H, hidden_mlp_dim, inp_features, out_features, dropout_rate, pred_len, embed_model ):
        super(Transformer, self).__init__()
        self.sqrt_D = torch.tensor(math.sqrt(D))
        self.num_layers = num_layers
        self.embed_model = embed_model
        if embed_model == 'LSTM':
          self.input_projection = nn.LSTM(inp_features, int(D/2), bidirectional=True) # multivariate input
        else:
          self.input_projection = nn.Linear(inp_features, D)
        self.Denoising=DenoisingAutoencoder(D)
        self.fitureSelection = nn.Linear(inp_features, inp_features) #feature selection
        self.output_projection = nn.Linear(D, out_features)  # multivariate output
        self.pos_encoding = positional_encoding(D)
        self.dec_layers = nn.ModuleList([TransformerLayer(D, H, hidden_mlp_dim,
                                                          dropout_rate=dropout_rate
                                                          ) for _ in range(num_layers)])
        self.dropout = nn.Dropout(dropout_rate)
        self.pred_len = pred_len
        self.D = D

    def forward(self, x, mask):

        B, S, D = x.shape
        attention_weights = {}
        x = nn.GELU()(self.fitureSelection(x))
        if self.embed_model == 'LSTM':
          hidden = torch.zeros((2, B, int(self.D/2)), dtype=torch.double)
          x, _ = self.input_projection(x.permute(1, 0, 2), (hidden, hidden))
          x = x.permute(1, 0, 2)
 
        else:
          x = self.input_projection(x)

        x = self.Denoising(x)
        x *= self.sqrt_D
        x += self.pos_encoding[:, :S, :]

        x = self.dropout(x)

        for i in range(self.num_layers):
            x, block = self.dec_layers[i](x=x,
                                          look_ahead_mask=mask)
            attention_weights['decoder_layer{}'.format(i + 1)] = block

        x = self.output_projection(x)
        x = x[:, -self.pred_len:, :]

        return x, attention_weights  # (B,S,S)


# Test Forward pass on the Transformer:
pred_len = 24

#creating list
combinations = [[3,64,1,100],[1,64,1,10],[3,32,1,50],[1,64,1,50],[3,32,1,10]]

from tqdm import tqdm

conf_ls = {}

for i, config in enumerate(combinations):
    start_time = time.time()
    
    n_layers, d_model, head, epoch = config
    niter = len(train_dataset)
    losses, val_losses = [], []

    transformer = Transformer(num_layers=n_layers, D=d_model, H=head, hidden_mlp_dim=d_model, 
                              inp_features=2, out_features=1, dropout_rate=0.1, pred_len=pred_len, embed_model = 'LSTM').double()
    transformer.to(device)

    optimizer = torch.optim.RMSprop(transformer.parameters(),
                                lr=0.00005)

    best_model = transformer

    val_loss = 1e5

    for e in tqdm(range(epoch)):

        # one epoch on train set
        transformer.train()
        sum_train_loss = 0.0
        for x, y in train_dataset:

            S = x.shape[1]
            y = y[:, 0:pred_len, :]
            mask = create_look_ahead_mask(S)
            out, _ = transformer(x, mask)
            loss = torch.nn.MSELoss()(out, y)
            sum_train_loss += loss.item()
            loss.backward()
            optimizer.step()
        losses.append(sum_train_loss / niter)

        # Evaluate on val set
        transformer.eval()
        sum_val_loss = 0.0
        for i, (x, y) in enumerate(val_dataset):
            S = x.shape[1]
            mask = create_look_ahead_mask(S)
            out, _ = transformer(x, mask)
            loss = torch.nn.MSELoss()(out, y)
            sum_val_loss += loss.item()
            if sum_val_loss < val_loss:
              val_loss = sum_val_loss
              best_model = transformer
            
        val_losses.append(sum_val_loss / (i + 1))

    test_losses, test_preds = [], []
    transformer.eval()
    for (x, y) in test_dataset:
        S = x.shape[-2]
        y_pred, _ = transformer(x,
                                mask=create_look_ahead_mask(S))
        loss_test = torch.nn.MSELoss()(y_pred, y)  # (B,S)
        test_losses.append(loss_test.item())
        test_preds.append(y_pred.detach().cpu().numpy())
    
    test_preds = np.vstack(test_preds)
    np.mean(test_losses)

    _, y_test = test_dataset.dataset.tensors

    y_test = y_test[1, :, 0].cpu().numpy()
    pred = test_preds[1, :, 0]

    MSE = np.square(np.subtract(y_test,pred)).mean() 
    RMSE = math.sqrt(MSE) #RMSE

    MAE = np.mean(np.abs(pred - y_test))    # MAE

    conf_ls["{},{},{},{}".format(n_layers, d_model, head, epoch)] = (RMSE, MAE)
    end_time = time.time()
    runtime = end_time - start_time
    print("Runtime: ", runtime, "seconds")
    

def RMSELoss(y_pred, y):
    return torch.sqrt(torch.mean((y_pred - y) ** 2))


criterion = RMSELoss
loss = criterion(y_pred, y)



def MAPELoss(y_pred, y):
    return torch.mean(torch.abs((y - y_pred) / y))


criterion = MAPELoss
loss = criterion(y_pred, y)


x = np.linspace(1, pred_len, pred_len)

data_mean, data_std = stats 
pred_BT = test_preds*data_std[0]+ data_mean[0]
y = y_test*data_std[0] + data_mean[0]

print(conf_ls)
plt.plot(y, color='blue',marker='o', markerfacecolor='blue', markersize=5,)
plt.plot(pred_BT[1], alpha=0.7, marker = '+',markersize=5, color='orange', linewidth = '5')


  0%|          | 0/100 [00:00<?, ?it/s]


RuntimeError: ignored

RNN



In [ ]:
from keras.callbacks import History

trainX, trainY = train_dataset.dataset.tensors[0].detach().numpy(), train_dataset.dataset.tensors[1].detach().numpy()
trainY = trainY[:, 0:pred_len, :]
testX, testY = test_dataset.dataset.tensors[0].detach().numpy(), test_dataset.dataset.tensors[1].detach().numpy()
testY = testY[:, 0:pred_len, :]
x_val, y_val = val_dataset.dataset.tensors[0].detach().numpy(), val_dataset.dataset.tensors[1].detach().numpy()
y_val = y_val[:, 0:pred_len, :]


def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape, 
                        activation=activation[0]))
    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

for i, config in enumerate(combinations):
    
    n_layers, d_model, head, epoch = config

    niter = len(train_dataset)
    model = create_RNN(hidden_units=d_model, dense_units = pred_len, input_shape =(trainX.shape[1], trainX.shape[2]),  
                   activation=['tanh', 'tanh'])
                   
    model_fit = model.fit(trainX, trainY, epochs = epoch, batch_size=64, verbose=2, validation_data=(x_val,y_val))
    model_output = model.predict(testX)
    results = pd.DataFrame({"Train Loss": model_fit.history['loss'], "Validation Loss": model_fit.history['val_loss']})
    #results[["Train results", "Validation Loss"]].plot()
    testY = testY.reshape(testY.shape[0], testY.shape[1], )


    #MAE and RMSE
    MSE = np.square(np.subtract(testY,model_output)).mean() 
    RMSE = np.sqrt(MSE)

    mae = np.mean(np.abs(model_output - testY))    # MAE
    conf_ls["{},{},{},{}".format(n_layers, d_model, head, epoch)] = (RMSE, mae)

prediction_RNN = model_output

data_mean, data_std = stats 
prediction_RNN = prediction_RNN*data_std[0]+ data_mean[0]
testY = testY*data_std[0] + data_mean[0]
print(conf_ls)


LSTM

In [ ]:

trainX, trainY = train_dataset.dataset.tensors[0].detach().numpy(), train_dataset.dataset.tensors[1].detach().numpy()
trainY = trainY[:, 0:pred_len, :]
testX, testY = test_dataset.dataset.tensors[0].detach().numpy(), test_dataset.dataset.tensors[1].detach().numpy()
testY = testY[:, 0:pred_len, :]
x_val, y_val = val_dataset.dataset.tensors[0].detach().numpy(), val_dataset.dataset.tensors[1].detach().numpy()
y_val = y_val[:, 0:pred_len, :]
# reshape input to be [samples, time steps, features]

for i, config in enumerate(combinations):
    
    n_layers, d_model, head, epoch = config
    niter = len(train_dataset)
    model = Sequential()
    model.add(LSTM(d_model, input_shape=(trainX.shape[1], trainX.shape[2])))
    model.add(Dense(pred_len, activation='linear'))
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    model_fit = model.fit(trainX, trainY, epochs = epoch, batch_size=64,  validation_data=(x_val,y_val))
    model_output = model.predict(testX)
    resultsLSTM = pd.DataFrame({"Train Loss": model_fit.history['loss'], "Validation Loss": model_fit.history['val_loss']})
    testY = testY.reshape(testY.shape[0], testY.shape[1], )

    MSE = np.square(np.subtract(testY,model_output)).mean() 
    RMSE = math.sqrt(MSE)

    mae = np.mean(np.abs(model_output - testY))    # MAE
    conf_ls["{},{},{},{}".format(n_layers, d_model, head, epoch)] = (RMSE, mae)

prediction_LSTM = model_output
data_mean, data_std = stats 
prediction_LSTM = prediction_LSTM*data_std[0]+ data_mean[0]
testY = testY*data_std[0] + data_mean[0]
print(conf_ls)

GRU

In [ ]:
from keras.layers import GRU
trainX, trainY = train_dataset.dataset.tensors[0].detach().numpy(), train_dataset.dataset.tensors[1].detach().numpy()
trainY = trainY[:, 0:pred_len, :]
testX, testY = test_dataset.dataset.tensors[0].detach().numpy(), test_dataset.dataset.tensors[1].detach().numpy()
testY = testY[:, 0:pred_len, :]
x_val, y_val = val_dataset.dataset.tensors[0].detach().numpy(), val_dataset.dataset.tensors[1].detach().numpy()
y_val = y_val[:, 0:pred_len, :]
# reshape input to be [samples, time steps, features]

for i, config in enumerate(combinations):
    
    n_layers, d_model, head, epoch = config
    niter = len(train_dataset)
    model = Sequential()
    model.add(GRU(d_model, input_shape=(trainX.shape[1], trainX.shape[2])))
    model.add(Dense(pred_len, activation='linear'))
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    model_fit = model.fit(trainX, trainY, epochs = epoch, batch_size=64,  validation_data=(x_val,y_val))
    model_output = model.predict(testX)
    resultsGRU = pd.DataFrame({"Train Loss": model_fit.history['loss'], "Validation Loss": model_fit.history['val_loss']})
    resultsGRU[["Train Loss", "Validation Loss"]].plot()
    testY = testY.reshape(testY.shape[0], testY.shape[1], )

    MSE = np.square(np.subtract(testY,model_output)).mean() 
    RMSE = math.sqrt(MSE)

    mae = np.mean(np.abs(model_output - testY))    # MAE
    conf_ls["{},{},{},{}".format(n_layers, d_model, head, epoch)] = (RMSE, mae)
prediction_GRU = model_output
data_mean, data_std = stats 
prediction_GRU = prediction_GRU*data_std[0]+ data_mean[0]
testY = testY*data_std[0] + data_mean[0]
print(conf_ls)

CNN

In [ ]:
from keras.layers import Dense, Conv1D, Flatten
from keras.layers.convolutional import Conv1D
trainX, trainY = train_dataset.dataset.tensors[0].detach().numpy(), train_dataset.dataset.tensors[1].detach().numpy()
trainY = trainY[:, 0:pred_len, :]
testX, testY = test_dataset.dataset.tensors[0].detach().numpy(), test_dataset.dataset.tensors[1].detach().numpy()
testY = testY[:, 0:pred_len, :]
x_val, y_val = val_dataset.dataset.tensors[0].detach().numpy(), val_dataset.dataset.tensors[1].detach().numpy()
y_val = y_val[:, 0:pred_len, :]
# reshape input to be [samples, time steps, features]

for i, config in enumerate(combinations):
    
    n_layers, d_model, head, epoch = config
    niter = len(train_dataset)
    model = Sequential()
    model.add(Conv1D(d_model, kernel_size=2, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2])))
    model.add(Flatten())
    model.add(Dense(pred_len, activation='relu'))
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    model_fit = model.fit(trainX, trainY, epochs = epoch, batch_size=64,  validation_data=(x_val,y_val))
    model_output = model.predict(testX)
    resultsCNN = pd.DataFrame({"Train Loss": model_fit.history['loss'], "Validation Loss": model_fit.history['val_loss']})
    resultsCNN[["Train Loss", "Validation Loss"]].plot()
    testY = testY.reshape(testY.shape[0], testY.shape[1], )

    MSE = np.square(np.subtract(testY,model_output)).mean() 
    RMSE = math.sqrt(MSE)

    mae = np.mean(np.abs(model_output - testY))    # MAE
    conf_ls["{},{},{},{}".format(n_layers, d_model, head, epoch)] = (RMSE, mae)
prediction_CNN= model_output
data_mean, data_std = stats 
prediction_CNN = prediction_CNN*data_std[0]+ data_mean[0]
testY = testY*data_std[0] + data_mean[0]
print(conf_ls)
print(conf_ls)

In [ ]:
plt.plot(y, color='blue',marker='o', markerfacecolor='blue', markersize=5,)
plt.plot(pred_T[1], alpha=0.7, marker = '+',markersize=5, color='orange', linewidth = '5')
plt.plot(pred_BT[1], alpha=0.7, marker = '+',markersize=5, color='black', linewidth = '5', LABLE = 'DABT')
plt.plot(prediction_RNN[1], alpha=0.7, color='yellow',label='RNN')
plt.plot(prediction_LSTM[1], linestyle='dashed', linewidth=3, label='LSTM')
plt.plot(prediction_GRU[1],  linewidth=3, label='CNN')
plt.plot(prediction_CNN[1],  linewidth=3, label='GRU')
plt.xlabel('Time', fontsize = 15)
plt.ylabel(' EV charging Load', fontsize = 15)
plt.legend(['Transformer','DABT' 'RNN', 'LSTM', 'CNN' ,'GRU'], fontsize = 15)

plt.show()